In [1]:
import cards

In [2]:
decklist = """
1 Arboreal Grazer
1 Llanowar Elves
1 Elvish Mystic
4 Sakura-Tribe Elder
4 Wall of Roots
4 Chancellor of the Tangle
1 Panglacial Wurm
1 Abundant Harvest
4 Ancient Stirrings
4 Caravan Vigil
2 Lay of the Land
4 Reclaim the Wastes
3 Explore
4 Land Grant
3 Rampant Growth
2 Nissa's Pilgrimage
4 Recross the Paths
2 Search for Tomorrow
4 Goblin Charbelcher
4 Wild Growth
7 Forest
"""

In [3]:
# Create a player with a predictable seed
player = cards.Player(decklist, 42)
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get and play a land
land = player.debug_force_get_card_in_hand('Forest')
player.play(land)

# Get a Search for Tomorrow
search = player.debug_force_get_card_in_hand("Search for Tomorrow")

# Cast the card for its alternate cost
player.alt_play(search)

# Pass the turn
player.start_turn()
# Ensure that we still only have 1 land
assert player.lands == 1
assert player.mana_pool == 1

# Pass the turn again
player.start_turn()
# Ensure that we now have 2 lands, both untapped
assert player.lands == 2
assert player.mana_pool == 2



TypeError: object of type 'int' has no len()